In [106]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy
import combine_svm
import pandas as pd
from tqdm import tqdm
import scipy.signal as signal
import csv
import seaborn as sns
sns.set()


### Function

In [107]:
''' ----------------------- Movement ----------------------- '''
# 30s訊號 = 30 * 20 = 600
def mov_dens_fn(raw_sig):
    new_sig = []
    count = 0
    for num in range(80):
        top = 0
        first = int(num*(0.5*20))
        last = int((num+1)*(0.5*20))
        x = raw_sig[first:last]

        # 方差公式
        for i in range(10):
            top += np.square(x[i] - np.average(x))
        result = top / (10 - 1)
        if result > 0.005:  # 閥值可調
            count += 1
    percent = (count/80) * 100
    return percent

''' ----------------------- Respiration ----------------------- '''
# 10個fRSA
def tfRSA_fn(fRSA_sig):
    tfRSA = np.std(fRSA_sig)
    return tfRSA

# 31個f𝑅𝑆𝐴
def sfRSA_fn(fRSA_sig):
    sfRSA = scipy.signal.savgol_filter(fRSA_sig, 31, 3)
    sfRSA_mean = np.average(sfRSA)
    return sfRSA, sfRSA_mean

# 31個tf𝑅𝑆𝐴
def stfRSA_fn(tfRSA_sig):
    stfRSA = scipy.signal.savgol_filter(np.array(tfRSA_sig), 31, 2)
    stfRSA_mean = np.average(stfRSA)
    return stfRSA, stfRSA_mean

# 31個f𝑅𝑆𝐴
def sdfRSA_fn(fRSA, sfRSA):
    sdfRSA = np.abs(fRSA - sfRSA)
    sdfRSA = scipy.signal.savgol_filter(sdfRSA, 31, 3)
    sdfRSA_mean = np.average(sdfRSA)
    return sdfRSA, sdfRSA_mean

''' ----------------------- Heart rate ----------------------- '''
def tmHR_fn(mHR_sig):
    return tfRSA_fn(mHR_sig)

def smHR_fn(mHR_sig):
    return sfRSA_fn(mHR_sig)

def stmHR_fn(tmHR_sig):
    return stfRSA_fn(tmHR_sig)

def sdmHR_fn(mHR, smHR):
    return sdfRSA_fn(mHR, smHR)

# ---------------- 待補 ---------------- 
# def LF_HF_LFHF(sig):
#     LF_sig = combine_svm.iir_bandpass_filter_1(amp_sig, 0.04, 0.15, 20, 9, "cheby2")
#     HF_sig = combine_svm.iir_bandpass_filter_1(amp_sig, 0.15, 0.4, 20, 9, "cheby2")

# def sHF_fn(HF_sig):
#     sHF = scipy.signal.savgol_filter(HF_sig, 31, 3)
#     sHF_mean = np.average(sfRSA)
#     return sHF, sHF_mean

# def sLFHF_fn(LFHF_sig):
#     sLFHF = scipy.signal.savgol_filter(LFHF_sig, 31, 3)
#     sLFHF_mean = np.average(sfRSA)
#     return sLFHF, sLFHF_mean


### 讀所有資料

In [108]:
names = "./dataset_sleep"
for name in os.listdir(names):
    files = os.path.join(names, name, "0.8")
    for num in range(len(os.listdir(files)) // 2):
        datas = os.listdir(files)[num]
        data = os.path.join(files, datas)
        raw_data = pd.read_csv(data)
        raw_data_pd = pd.DataFrame(raw_data)
raw_data_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224099 entries, 0 to 224098
Data columns (total 39 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   rangeBinIndexMax                 224099 non-null  int64  
 1   rangeBinIndexPhase               224099 non-null  int64  
 2   maxVal                           224099 non-null  int64  
 3   processingCyclesOut              224099 non-null  int64  
 4   processingCyclesOut1             224099 non-null  int64  
 5   rangeBinStartIndex               224099 non-null  int64  
 6   rangeBinEndIndex                 224099 non-null  int64  
 7   unwrapPhasePeak_mm               224099 non-null  float64
 8   outputFilterBreathOut            224099 non-null  float64
 9   outputFilterHeartOut             224099 non-null  float64
 10  heartRateEst_FFT                 224099 non-null  float64
 11  heartRateEst_FFT_4Hz             224099 non-null  float64
 12  he

### 只取有心律跟呼吸律的時間點，把特徵單位轉成秒

In [109]:
# 特徵單位變成秒
raw_data_pd['heart'] = raw_data_pd['heart'].replace(0, np.nan)
new_data = raw_data_pd.dropna()
new_data.head(10)

,rangeBinIndexMax,rangeBinIndexPhase,maxVal,processingCyclesOut,processingCyclesOut1,rangeBinStartIndex,rangeBinEndIndex,unwrapPhasePeak_mm,outputFilterBreathOut,outputFilterHeartOut,...,rsv[4],rsv[5],rsv[6],rsv[7],rsv[8],rsv[9],datetime,heart,breath,sleep
0,0,25,394851616,9247,0,11,73,115.318550,-0.150294,-0.082511,...,0,0,0,0,0,0,00:17:01,80.0,16,5
6,0,25,179700208,9250,0,11,73,114.844742,-0.134879,-0.144840,...,0,0,0,0,0,0,00:17:02,80.0,16,5
26,0,25,238785680,9247,0,11,73,116.224937,0.045472,-0.002680,...,0,0,0,0,0,0,00:17:03,79.0,16,5
46,0,25,529669856,9249,0,11,73,114.530541,-0.123384,-0.098686,...,0,0,0,0,0,0,00:17:04,79.0,16,5
66,0,25,379301344,9240,0,11,73,115.924042,0.053978,0.029496,...,0,0,0,0,0,0,00:17:05,80.0,16,5
86,0,25,517132128,9239,0,11,73,114.958298,-0.036623,0.067776,...,0,0,0,0,0,0,00:17:06,80.0,16,5
106,0,25,278349376,9246,0,11,73,115.764625,0.042648,0.042580,...,0,0,0,0,0,0,00:17:07,80.0,16,5
126,0,25,425000256,9251,0,11,73,115.576408,0.001536,0.080538,...,0,0,0,0,0,0,00:17:08,78.0,16,5
146,0,25,274435712,9250,0,11,73,115.179382,-0.027266,-0.137768,...,0,0,0,0,0,0,00:17:09,75.0,16,5
166,0,25,232315920,9253,0,11,73,116.542290,0.088777,0.077158,...,0,0,0,0,0,0,00:17:10,78.0,16,5


### 特徵: tfRSA, tmHR, sfRSA, smHR, sdfRSA, sdmHR

In [110]:
breath = np.array(new_data['breath'])
heart = np.array(new_data['heart'])
print(f"Total len: {len(heart)}\n")

# ------------- tfRSA and tmHR ------------- 
local_tfRSA = []
local_tmHR = []
num_in_window = 10

# 空值補0
for i in range(num_in_window-1):
    local_tfRSA.append(0)
    local_tmHR.append(0)

for turn in range(len(breath) - num_in_window + 1):
    start_index = turn
    end_index = start_index + num_in_window

    # Slide data
    window_breath = breath[start_index:end_index]
    window_heart = heart[start_index:end_index]

    # Standard deviation
    local_tfRSA.append(str(round(tfRSA_fn(window_breath), 4)))
    local_tmHR.append(str(round(tmHR_fn(window_heart), 4)))

print("tfRSA and tmHR Complete!")
print(f"Real len: {len(breath)}\ntfRSA len: {len(local_tfRSA)}\ntmHR len: {len(local_tmHR)}")

#  ------------- sfRSA and smHR and sdfRSA and sdmHR------------- 
local_sfRSA = []
local_sdfRSA = []
local_smHR = []
local_sdmHR = []
num_in_window = 31

# 空值補0
for i in range(num_in_window-1):
    local_sfRSA.append(0)
    local_smHR.append(0)
    local_sdfRSA.append(0)
    local_sdmHR.append(0)

for turn in range(len(breath) - num_in_window + 1):
    start_index = turn
    end_index = start_index + num_in_window

    # Slide data
    window_breath = breath[start_index:end_index]
    window_heart = heart[start_index:end_index]

    # Savitzky–Golay filter
    sfRSA, sfRSA_mean = sfRSA_fn(window_breath)
    sdfRSA, sdfRSA_mean = sdfRSA_fn(window_breath, sfRSA)
    smHR, smHR_mean = smHR_fn(window_heart)
    sdmHR, sdmHR_mean = sdmHR_fn(window_heart, smHR)
    local_sfRSA.append(round(sfRSA_mean, 4))
    local_sdfRSA.append(round(sdfRSA_mean, 4))
    local_smHR.append(round(smHR_mean, 4))
    local_sdmHR.append(round(sdmHR_mean, 4))

print("\nsfRSA and smHR Complete!")
print(f"Real len: {len(heart)}\nsfRSA len: {len(local_sfRSA)}\nsmHR len: {len(local_smHR)}")

# 插入特徵
new_data.insert(39, "tfRSA", local_tfRSA)
new_data.insert(40, "tmHR", local_tmHR)
new_data.insert(41, "sfRSA", local_sfRSA)
new_data.insert(42, "smHR", local_smHR)
new_data.insert(43, "sdfRSA", local_sdfRSA)
new_data.insert(44, "sdmHR", local_sdmHR)
new_data.head(10)

Total len: 8483

tfRSA and tmHR Complete!
Real len: 8483
tfRSA len: 8483
tmHR len: 8483

sfRSA and smHR Complete!
Real len: 8483
sfRSA len: 8483
smHR len: 8483


,rangeBinIndexMax,rangeBinIndexPhase,maxVal,processingCyclesOut,processingCyclesOut1,rangeBinStartIndex,rangeBinEndIndex,unwrapPhasePeak_mm,outputFilterBreathOut,outputFilterHeartOut,...,datetime,heart,breath,sleep,tfRSA,tmHR,sfRSA,smHR,sdfRSA,sdmHR
0,0,25,394851616,9247,0,11,73,115.318550,-0.150294,-0.082511,...,00:17:01,80.0,16,5,0,0,0.0,0.0,0.0,0.0
6,0,25,179700208,9250,0,11,73,114.844742,-0.134879,-0.144840,...,00:17:02,80.0,16,5,0,0,0.0,0.0,0.0,0.0
26,0,25,238785680,9247,0,11,73,116.224937,0.045472,-0.002680,...,00:17:03,79.0,16,5,0,0,0.0,0.0,0.0,0.0
46,0,25,529669856,9249,0,11,73,114.530541,-0.123384,-0.098686,...,00:17:04,79.0,16,5,0,0,0.0,0.0,0.0,0.0
66,0,25,379301344,9240,0,11,73,115.924042,0.053978,0.029496,...,00:17:05,80.0,16,5,0,0,0.0,0.0,0.0,0.0
86,0,25,517132128,9239,0,11,73,114.958298,-0.036623,0.067776,...,00:17:06,80.0,16,5,0,0,0.0,0.0,0.0,0.0
106,0,25,278349376,9246,0,11,73,115.764625,0.042648,0.042580,...,00:17:07,80.0,16,5,0,0,0.0,0.0,0.0,0.0
126,0,25,425000256,9251,0,11,73,115.576408,0.001536,0.080538,...,00:17:08,78.0,16,5,0,0,0.0,0.0,0.0,0.0
146,0,25,274435712,9250,0,11,73,115.179382,-0.027266,-0.137768,...,00:17:09,75.0,16,5,0,0,0.0,0.0,0.0,0.0
166,0,25,232315920,9253,0,11,73,116.542290,0.088777,0.077158,...,00:17:10,78.0,16,5,0.0,1.5133,0.0,0.0,0.0,0.0


### 為了要算 stfRSA and stmHR 將特徵單位變成tmHR

In [111]:
# 特徵單位變成tmHR
new_data['tmHR'] = new_data['tmHR'].replace(0, np.nan)
new_data = new_data.dropna()
new_data

C:\Users\story\AppData\Local\Temp/ipykernel_1900/2832778280.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['tmHR'] = new_data['tmHR'].replace(0, np.nan)


,rangeBinIndexMax,rangeBinIndexPhase,maxVal,processingCyclesOut,processingCyclesOut1,rangeBinStartIndex,rangeBinEndIndex,unwrapPhasePeak_mm,outputFilterBreathOut,outputFilterHeartOut,...,datetime,heart,breath,sleep,tfRSA,tmHR,sfRSA,smHR,sdfRSA,sdmHR
166,0,25,232315920,9253,0,11,73,116.542290,0.088777,0.077158,...,00:17:10,78.0,16,5,0.0,1.5133,0.0000,0.0000,0.0000,0.0000
186,0,25,338813504,9268,0,11,73,114.939575,-0.116598,-0.007121,...,00:17:11,74.0,16,5,0.0,2.0518,0.0000,0.0000,0.0000,0.0000
206,0,25,310181344,9252,0,11,73,116.689163,0.113466,-0.026823,...,00:17:12,73.0,16,5,0.0,2.498,0.0000,0.0000,0.0000,0.0000
226,0,25,439479200,9249,0,11,73,114.687088,-0.115219,-0.061958,...,00:17:13,75.0,16,5,0.0,2.5612,0.0000,0.0000,0.0000,0.0000
246,0,25,218203232,9248,0,11,73,116.352242,0.067727,0.046036,...,00:17:14,74.0,16,5,0.0,2.6476,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223999,0,27,150444368,9250,0,11,73,26.366610,-0.006233,-0.003249,...,03:23:42,83.0,17,4,0.0,3.0083,17.5484,80.9677,0.1992,3.1247
224019,0,27,141422480,9250,0,11,73,26.433498,0.001302,0.004042,...,03:23:43,82.0,17,4,0.0,2.9682,17.4839,81.0968,0.1848,3.0784
224039,0,27,130205040,9251,0,11,73,26.583200,0.002634,0.009661,...,03:23:44,81.0,17,4,0.0,2.9257,17.4194,81.1935,0.1452,3.0686
224059,0,23,144267024,9247,0,11,73,25.357492,-0.018730,-0.083931,...,03:23:45,80.0,16,4,0.3,2.9257,17.3548,81.2258,0.2114,3.0934


### 特徵: stfRSA and stmHR

In [112]:
tfRSA = np.array(new_data['tfRSA'])
tmHR = np.array(new_data['tmHR'])

#  ------------- stfRSA and stmHR ------------- 
local_stfRSA = []
local_stmHR = []
num_in_window = 31

# 空值補0
for i in range(num_in_window-1):
    local_stfRSA.append(0)
    local_stmHR.append(0)

for turn in range(len(tfRSA) - num_in_window + 1):
    start_index = turn
    end_index = start_index + num_in_window

    # Slide data
    window_tfRSA = tfRSA[start_index:end_index]
    window_tmHR = tmHR[start_index:end_index]

    # Savitzky–Golay filter
    stfRSA, stfRSA_mean = stfRSA_fn(window_tfRSA)
    stmHR, stmHR_mean = stmHR_fn(window_tmHR)
    local_stfRSA.append(round(stfRSA_mean, 4))
    local_stmHR.append(round(stmHR_mean, 4))

new_data.insert(45, "stfRSA", local_stfRSA)
new_data.insert(46, "stmHR", local_stmHR)

print("\nsfRSA and smHR Complete!")
print(f"Real len: {len(heart)}\nstfRSA len: {len(local_stfRSA)}\nstmHR len: {len(local_stmHR)}")


sfRSA and smHR Complete!
Real len: 8483
stfRSA len: 8474
stmHR len: 8474


### 只保留不為零的特徵

In [113]:
# 只保留不為零的特徵
new_data['stmHR'] = new_data['stmHR'].replace(0, np.nan)
new_data = new_data.dropna()
new_data

,rangeBinIndexMax,rangeBinIndexPhase,maxVal,processingCyclesOut,processingCyclesOut1,rangeBinStartIndex,rangeBinEndIndex,unwrapPhasePeak_mm,outputFilterBreathOut,outputFilterHeartOut,...,breath,sleep,tfRSA,tmHR,sfRSA,smHR,sdfRSA,sdmHR,stfRSA,stmHR
766,0,29,139441920,9241,0,11,73,101.757797,-0.039552,0.001218,...,17,5,0.5,1.1832,16.4194,72.9677,0.3285,3.3083,0.2240,3.5274
786,0,29,814841088,9239,0,11,73,101.806145,-0.006533,0.011618,...,17,5,0.5,1.3,16.4516,72.8065,0.3657,3.1806,0.2401,3.5205
806,0,29,128217584,9238,0,11,73,101.946175,0.008477,-0.016523,...,17,5,0.5,1.3,16.4839,72.8065,0.3762,3.0563,0.2563,3.4962
826,0,29,91835616,9258,0,11,73,101.858971,-0.000930,0.023020,...,17,5,0.5,1.2649,16.5161,72.8710,0.3685,2.7875,0.2724,3.4564
846,0,29,586722496,9238,0,11,73,102.031357,0.022816,-0.106009,...,17,5,0.4899,1.5524,16.5484,72.9677,0.3518,2.1850,0.2882,3.4239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223999,0,27,150444368,9250,0,11,73,26.366610,-0.006233,-0.003249,...,17,4,0.0,3.0083,17.5484,80.9677,0.1992,3.1247,0.2449,3.0098
224019,0,27,141422480,9250,0,11,73,26.433498,0.001302,0.004042,...,17,4,0.0,2.9682,17.4839,81.0968,0.1848,3.0784,0.2449,3.0848
224039,0,27,130205040,9251,0,11,73,26.583200,0.002634,0.009661,...,17,4,0.0,2.9257,17.4194,81.1935,0.1452,3.0686,0.2449,3.1578
224059,0,23,144267024,9247,0,11,73,25.357492,-0.018730,-0.083931,...,16,4,0.3,2.9257,17.3548,81.2258,0.2114,3.0934,0.2449,3.2308


In [114]:
new_data.to_csv("./sleep_features/test.csv", index=False)
print("Completed!")

Completed!
